In [ ]:
#!/usr/bin/env python3
"""
vLLM Tensor Parallelism Test with Auto File Detection
Tests bit-exact reproducibility across multiple runs with TP
Automatically finds txt/pdf files in current directory
Validates prompt length before loading model weights
Uses proper Kimi K2 chat template format
"""

import os
os.environ['HF_HOME'] = '/tmp/hf_cache'
os.environ['TRANSFORMERS_CACHE'] = '/tmp/hf_cache'

from vllm import LLM, SamplingParams
from transformers import AutoTokenizer
import numpy as np
from datetime import datetime
import json
import torch
import glob

# ============================================================================
# CONFIGURATION
# ============================================================================

# Model configuration
MODEL_NAME = "moonshotai/Kimi-K2-Thinking"
TENSOR_PARALLEL_SIZE = 8
MAX_MODEL_LEN = 150000
GPU_MEMORY_UTILIZATION = 0.9

# Generation configuration
MAX_TOKENS = 20
NUM_REPETITIONS = 20
TEMPERATURE = 0.0  # Greedy decoding
SEED = 42
TOP_LOGPROBS = 10

# Prompt source - finds first txt or pdf in current directory
AUTO_FIND_FILE = True  # Set to False to use hardcoded content

# Chat template configuration - using Kimi K2 defaults
SYSTEM_PROMPT = "You are Kimi, an AI assistant created by Moonshot AI."
USER_TASK = "Please provide a detailed summary of the following text."

# Hardcoded content (used if AUTO_FIND_FILE=False or no files found)
HARDCODED_CONTENT = """The development of large language models has fundamentally transformed natural language processing and artificial intelligence more broadly. These models, trained on vast corpora of text data, have demonstrated remarkable capabilities across a wide range of tasks, from translation and summarization to question answering and creative writing."""

# ============================================================================
# FILE LOADING UTILITIES
# ============================================================================

def find_prompt_file():
    """Find first txt or pdf file in current directory"""
    cwd = os.getcwd()
    
    # Look for txt files first, then pdf
    txt_files = glob.glob(os.path.join(cwd, "*.txt"))
    pdf_files = glob.glob(os.path.join(cwd, "*.pdf"))
    
    if txt_files:
        return txt_files[0]
    elif pdf_files:
        return pdf_files[0]
    else:
        return None

def load_text_from_file(filepath):
    """Load text from txt or pdf file"""
    if not os.path.exists(filepath):
        raise FileNotFoundError(f"File not found: {filepath}")
    
    if filepath.endswith('.txt'):
        with open(filepath, 'r', encoding='utf-8') as f:
            text = f.read()
        print(f"Loaded {len(text)} characters from txt file")
        return text
    
    elif filepath.endswith('.pdf'):
        try:
            import PyPDF2
        except ImportError:
            raise ImportError("PyPDF2 required for PDF loading. Install with: pip install PyPDF2")
        
        text = []
        with open(filepath, 'rb') as f:
            pdf_reader = PyPDF2.PdfReader(f)
            num_pages = len(pdf_reader.pages)
            print(f"Loading {num_pages} pages from PDF...")
            
            for page_num, page in enumerate(pdf_reader.pages, 1):
                page_text = page.extract_text()
                text.append(page_text)
                if page_num % 10 == 0:
                    print(f"  Processed {page_num}/{num_pages} pages")
        
        full_text = '\n'.join(text)
        print(f"Loaded {len(full_text)} characters from PDF ({num_pages} pages)")
        return full_text
    
    else:
        raise ValueError(f"Unsupported file type: {filepath}. Use .txt or .pdf")

# ============================================================================
# PROMPT LOADING
# ============================================================================

print("=" * 80)
print("vLLM LOGPROBS EXTRACTION TEST - KIMI K2")
print("=" * 80)
print()

# Load document content
prompt_file = None
if AUTO_FIND_FILE:
    prompt_file = find_prompt_file()
    if prompt_file:
        print(f"Found file: {os.path.basename(prompt_file)}")
        DOCUMENT_CONTENT = load_text_from_file(prompt_file)
        print()
    else:
        print("No txt/pdf files found in current directory")
        print("Using hardcoded content")
        DOCUMENT_CONTENT = HARDCODED_CONTENT
        print()
else:
    DOCUMENT_CONTENT = HARDCODED_CONTENT
    print("Using hardcoded content")
    print()

# Apply Kimi K2 chat template
# Based on official template: https://huggingface.co/moonshotai/Kimi-K2-Thinking/blob/main/chat_template.jinja
print("Applying Kimi K2 chat template...")

# Combine user task with document
user_content = f"{USER_TASK}\n\n{DOCUMENT_CONTENT}"

# Format following Kimi K2 template structure:
# 1. System message with default Kimi prompt
# 2. User message
# 3. Assistant start token (model will generate after this)
PROMPT = (
    f"<|im_system|>system<|im_middle|>{SYSTEM_PROMPT}<|im_end|>"
    f"<|im_user|>user<|im_middle|>{user_content}<|im_end|>"
    f"<|im_assistant|>assistant<|im_middle|>"
)

print(f"Formatted prompt length: {len(PROMPT)} characters")
print()

# ============================================================================
# TOKENIZER PRECHECK
# ============================================================================

print("Loading tokenizer to validate prompt length...")
tokenizer = AutoTokenizer.from_pretrained(
    MODEL_NAME,
    cache_dir='/tmp/hf_cache',
    trust_remote_code=True
)

# Tokenize prompt
prompt_tokens = tokenizer.encode(PROMPT)
prompt_length = len(prompt_tokens)

print(f"Prompt statistics:")
print(f"  Characters: {len(PROMPT):,}")
print(f"  Tokens: {prompt_length:,}")
print(f"  Max model length: {MAX_MODEL_LEN:,}")
print(f"  Generation tokens: {MAX_TOKENS}")
print(f"  Total required: {prompt_length + MAX_TOKENS:,}")
print()

# Validate length
if prompt_length > MAX_MODEL_LEN:
    print(f"❌ ERROR: Prompt is too long!")
    print(f"  Prompt has {prompt_length:,} tokens")
    print(f"  Model max is {MAX_MODEL_LEN:,} tokens")
    print(f"  Exceeds by {prompt_length - MAX_MODEL_LEN:,} tokens")
    print()
    print("Solutions:")
    print(f"  1. Increase MAX_MODEL_LEN to at least {prompt_length + MAX_TOKENS}")
    print(f"  2. Truncate/reduce the prompt")
    exit(1)

if prompt_length + MAX_TOKENS > MAX_MODEL_LEN:
    print(f"⚠ WARNING: Prompt + generation may exceed context")
    print(f"  Prompt: {prompt_length:,} tokens")
    print(f"  Generation: {MAX_TOKENS} tokens")
    print(f"  Total: {prompt_length + MAX_TOKENS:,} tokens")
    print(f"  Model max: {MAX_MODEL_LEN:,} tokens")
    print(f"  Consider increasing MAX_MODEL_LEN to {prompt_length + MAX_TOKENS + 100}")
    print()
else:
    print(f"✓ Prompt length validation passed")
    print(f"  Remaining capacity: {MAX_MODEL_LEN - prompt_length - MAX_TOKENS:,} tokens")
    print()

# ============================================================================
# VLLM PROMPT TEMPLATE HANDLING
# ============================================================================

print("Note on prompt formatting:")
print("  Using OFFICIAL Kimi K2 chat template")
print("  Template structure:")
print("    - <|im_system|>system<|im_middle|>...<|im_end|>")
print("    - <|im_user|>user<|im_middle|>...<|im_end|>")
print("    - <|im_assistant|>assistant<|im_middle|> (generation start)")
print("  Why manual template:")
print("    - Full control over exact formatting")
print("    - Reproducible across environments")
print("    - Matches model's training format")
print()

# ============================================================================
# MODEL LOADING
# ============================================================================

print("Configuration:")
print(f"  Model: {MODEL_NAME}")
print(f"  Tensor parallel: {TENSOR_PARALLEL_SIZE}")
print(f"  Max model len: {MAX_MODEL_LEN:,}")
print(f"  GPU memory utilization: {GPU_MEMORY_UTILIZATION} ({int(GPU_MEMORY_UTILIZATION*100)}%)")
print(f"  Max tokens: {MAX_TOKENS}")
print(f"  Repetitions: {NUM_REPETITIONS}")
print(f"  Temperature: {TEMPERATURE} (greedy)")
print(f"  Top logprobs: {TOP_LOGPROBS}")
print()

print("GPU Info:")
print(f"  Device: {torch.cuda.get_device_name(0)}")
print(f"  Available: {torch.cuda.device_count()} GPUs")
print()

print("Loading model with vLLM...")
llm = LLM(
    model=MODEL_NAME,
    trust_remote_code=True,
    download_dir='/tmp/hf_cache',
    dtype='bfloat16',
    max_model_len=MAX_MODEL_LEN,
    tensor_parallel_size=TENSOR_PARALLEL_SIZE,
    enable_prefix_caching=False,
    gpu_memory_utilization=GPU_MEMORY_UTILIZATION,
    disable_log_stats=True
)
print("Model loaded successfully!")
print()

# Sampling parameters
sampling_params = SamplingParams(
    temperature=TEMPERATURE,
    max_tokens=MAX_TOKENS,
    seed=SEED,
    logprobs=TOP_LOGPROBS,
    prompt_logprobs=None
)

# Warmup run
print("Running warmup...")
warmup_output = llm.generate([PROMPT], sampling_params=sampling_params)
print("Warmup complete")
print()

# ============================================================================
# TEST RUNS
# ============================================================================

print("=" * 80)
print(f"Running {NUM_REPETITIONS} test repetitions")
print("=" * 80)
print()

results_tokens = []
results_logprobs = []
results_texts = []
results_distributions = []

for rep in range(NUM_REPETITIONS):
    print(f"Repetition {rep + 1}/{NUM_REPETITIONS}...")
    
    outputs = llm.generate([PROMPT], sampling_params=sampling_params)
    output = outputs[0]
    
    # Extract token IDs
    token_ids = output.outputs[0].token_ids
    results_tokens.append(token_ids)
    
    # Extract generated text
    text = output.outputs[0].text
    results_texts.append(text)
    
    # Extract logprobs for selected tokens
    logprobs_data = output.outputs[0].logprobs
    selected_logprobs = [lp[token_ids[i]].logprob for i, lp in enumerate(logprobs_data)]
    results_logprobs.append(np.array(selected_logprobs))
    
    # Extract full top-k distributions
    rep_distributions = []
    for position_logprobs in logprobs_data:
        # Get top-k sorted by logprob (descending)
        sorted_items = sorted(position_logprobs.items(), 
                            key=lambda x: x[1].logprob, 
                            reverse=True)[:TOP_LOGPROBS]
        rep_distributions.append([(tok, lp.logprob) for tok, lp in sorted_items])
    results_distributions.append(rep_distributions)
    
    print(f"  Generated {len(token_ids)} tokens")

print()
print("All repetitions complete!")
print()

# ============================================================================
# ANALYSIS
# ============================================================================

print("=" * 80)
print("ANALYSIS")
print("=" * 80)
print()

# Check token sequence identity
print("Checking token sequences...")
tokens_identical = all(
    results_tokens[0] == results_tokens[i] 
    for i in range(1, NUM_REPETITIONS)
)
print(f"Token sequences identical: {tokens_identical}")

if not tokens_identical:
    print("\n⚠ Token sequences differ!")
    for i in range(1, NUM_REPETITIONS):
        if results_tokens[0] != results_tokens[i]:
            diff_positions = [
                j for j in range(min(len(results_tokens[0]), len(results_tokens[i])))
                if results_tokens[0][j] != results_tokens[i][j]
            ]
            print(f"  Rep 0 vs Rep {i}: {len(diff_positions)} positions differ")
            if diff_positions:
                print(f"    First difference at position {diff_positions[0]}")

# Check logprobs for selected tokens
print("\nChecking selected token logprobs...")
first_logprobs = results_logprobs[0]
logprobs_exact = all(
    np.allclose(first_logprobs, results_logprobs[i], rtol=0, atol=1e-10)
    for i in range(1, NUM_REPETITIONS)
)
print(f"Selected token logprobs bit-exact: {logprobs_exact}")

# Check top-k distributions
print("\nChecking full top-k distributions...")
distributions_exact = True
distribution_mismatches = []

first_dist = results_distributions[0]
for rep_idx in range(1, NUM_REPETITIONS):
    for pos_idx in range(len(first_dist)):
        dist_a = first_dist[pos_idx]
        dist_b = results_distributions[rep_idx][pos_idx]
        
        # Check if token IDs match in same order
        tokens_match = [t[0] for t in dist_a] == [t[0] for t in dist_b]
        
        # Check if logprobs are bit-exact
        if tokens_match:
            logprobs_match = all(
                abs(dist_a[i][1] - dist_b[i][1]) < 1e-10 
                for i in range(len(dist_a))
            )
            if not logprobs_match:
                distributions_exact = False
                distribution_mismatches.append((rep_idx, pos_idx))
        else:
            distributions_exact = False
            distribution_mismatches.append((rep_idx, pos_idx))

print(f"Top-k distributions bit-exact: {distributions_exact}")

if not distributions_exact:
    print(f"\n⚠ Found {len(distribution_mismatches)} position mismatches in distributions")
    if len(distribution_mismatches) <= 5:
        for rep_idx, pos_idx in distribution_mismatches:
            print(f"  Rep 0 vs Rep {rep_idx}, position {pos_idx}")
    else:
        print(f"  First 5: {distribution_mismatches[:5]}")

if not logprobs_exact:
    print("\nL2 distances:")
    l2_distances = []
    for i in range(1, NUM_REPETITIONS):
        l2 = np.linalg.norm(first_logprobs - results_logprobs[i])
        l2_distances.append(l2)
        print(f"  Rep 0 vs Rep {i}: L2 = {l2:.6e}")
    
    print(f"\nMax L2: {max(l2_distances):.6e}")
    print(f"Mean L2: {np.mean(l2_distances):.6e}")
    
    # Element-wise statistics
    all_logprobs = np.array(results_logprobs)
    std_per_token = all_logprobs.std(axis=0)
    print(f"\nPer-token std statistics:")
    print(f"  Mean: {std_per_token.mean():.6e}")
    print(f"  Max: {std_per_token.max():.6e}")
    print(f"  Median: {np.median(std_per_token):.6e}")

print()

# ============================================================================
# VERDICT
# ============================================================================

print("=" * 80)
print("VERDICT")
print("=" * 80)
print()

if tokens_identical and logprobs_exact and distributions_exact:
    print("✓ PERFECT REPRODUCIBILITY")
    print("  - Token sequences: bit-exact")
    print("  - Selected token logprobs: bit-exact")
    print("  - Full top-k distributions: bit-exact")
    print("  - vLLM is deterministic for this config")
elif tokens_identical and logprobs_exact and not distributions_exact:
    print("⚠ SELECTED TOKENS EXACT, DISTRIBUTIONS VARY")
    print("  - Token sequences: bit-exact")
    print("  - Selected token logprobs: bit-exact")
    print("  - Top-k distributions: numerical variation detected")
    print("  → May indicate computational instability in non-selected paths")
elif tokens_identical and not logprobs_exact:
    print("⚠ TOKENS IDENTICAL, LOGPROBS VARY")
    print("  - Token sequences: bit-exact")
    print("  - Logprobs: small numerical variation")
    max_l2 = max(l2_distances) if not logprobs_exact else 0.0
    if max_l2 < 1e-6:
        print(f"  - Variation very small (L2={max_l2:.2e})")
        print("  → Likely acceptable for forensics")
    else:
        print(f"  - Variation notable (L2={max_l2:.2e})")
        print("  → Investigate noise source")
else:
    print("✗ TOKEN SEQUENCES DIFFER")
    print("  - This should NOT happen with temperature=0")
    print("  → Something is wrong, investigate")

print()

# ============================================================================
# SAVE RESULTS
# ============================================================================

output_data = {
    "experiment": "vllm_kimi_k2_logprobs_test",
    "timestamp": datetime.now().isoformat(),
    "prompt_source": "file" if prompt_file else "hardcoded",
    "prompt_file": os.path.basename(prompt_file) if prompt_file else None,
    "prompt_length_chars": len(PROMPT),
    "prompt_length_tokens": prompt_length,
    "chat_template": "kimi_k2_official",
    "config": {
        "model": MODEL_NAME,
        "tensor_parallel": TENSOR_PARALLEL_SIZE,
        "max_model_len": MAX_MODEL_LEN,
        "max_tokens": MAX_TOKENS,
        "repetitions": NUM_REPETITIONS,
        "temperature": TEMPERATURE,
        "seed": SEED,
        "warmup_enabled": True,
        "prefix_caching_disabled": True,
        "top_logprobs": TOP_LOGPROBS
    },
    "results": {
        "tokens_identical": tokens_identical,
        "logprobs_exact": logprobs_exact,
        "distributions_exact": distributions_exact,
        "perfect_reproducibility": tokens_identical and logprobs_exact and distributions_exact
    },
    "token_sequences": results_tokens,
    "logprobs_vectors": [lp.tolist() for lp in results_logprobs],
    "generated_texts": results_texts,
    "top_k_distributions": [
        [[(int(tok), float(prob)) for tok, prob in dist] for dist in rep_dists]
        for rep_dists in results_distributions
    ]
}

output_file = f"vllm_kimi_k2_test_{datetime.now().strftime('%Y%m%d_%H%M%S')}.json"
with open(output_file, "w") as f:
    json.dump(output_data, f, indent=2)

print(f"Results saved to: {output_file}")
print()
print("=" * 80)
print("TEST COMPLETE")
print("=" * 80)
print()

vLLM LOGPROBS EXTRACTION TEST - KIMI K2

Found file: Verification-for-International-AI-Governance.pdf
Loading 172 pages from PDF...
  Processed 10/172 pages
  Processed 20/172 pages
  Processed 30/172 pages
  Processed 40/172 pages
  Processed 50/172 pages
  Processed 60/172 pages
  Processed 70/172 pages
  Processed 80/172 pages
  Processed 90/172 pages
  Processed 100/172 pages
  Processed 110/172 pages
  Processed 120/172 pages
  Processed 130/172 pages
  Processed 140/172 pages
  Processed 150/172 pages
  Processed 160/172 pages
  Processed 170/172 pages
Loaded 535619 characters from PDF (172 pages)

Applying Kimi K2 chat template...
Formatted prompt length: 535848 characters

Loading tokenizer to validate prompt length...
Prompt statistics:
  Characters: 535,848
  Tokens: 115,730
  Max model length: 150,000
  Generation tokens: 20
  Total required: 115,750

✓ Prompt length validation passed
  Remaining capacity: 34,250 tokens

Note on prompt formatting:
  Using OFFICIAL Kimi K2 ch

The argument `trust_remote_code` is to be used with Auto classes. It has no effect here and is ignored.
A new version of the following files was downloaded from https://huggingface.co/moonshotai/Kimi-K2-Thinking:
- configuration_deepseek.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.
You are using a model of type kimi_k2 to instantiate a model of type deepseek_v3. This is not supported for all configurations of models and can yield errors.


INFO 11-10 02:31:59 [config.py:388] Replacing legacy 'type' key with 'rope_type'
INFO 11-10 02:31:59 [model.py:547] Resolved architecture: DeepseekV3ForCausalLM
INFO 11-10 02:31:59 [model.py:1510] Using max model len 150000
INFO 11-10 02:31:59 [scheduler.py:205] Chunked prefill is enabled with max_num_batched_tokens=16384.
INFO 11-10 02:31:59 [cuda.py:166] Forcing kv cache block size to 64 for FlashMLA backend.


A new version of the following files was downloaded from https://huggingface.co/moonshotai/Kimi-K2-Thinking:
- tokenization_kimi.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


WARNING 11-10 02:32:00 [tokenizer.py:253] Using a slow tokenizer. This might cause a significant slowdown. Consider using a fast tokenizer instead.


...

Processed prompts:   0%|          | 0/1 [00:00<?, ?it/s, est. speed input: 0.00 toks/s, output: 0.00 toks/s]

  Generated 20 tokens
Repetition 19/20...


Adding requests:   0%|          | 0/1 [00:00<?, ?it/s]

Calling super().encode with {'truncation': False}


Processed prompts:   0%|          | 0/1 [00:00<?, ?it/s, est. speed input: 0.00 toks/s, output: 0.00 toks/s]